#### Primeiramente os dados dos arquivos csv foram importados para um dataframe do pandas e, em seguida, foram ajustados com os nomes de colunas apropriados para uma manipulação mais fácil.

In [98]:
import pandas as pd
import sqlite3 as sql
import os

In [106]:
lista_csv = os.listdir("/home/michelrribeiro/virtualpython/projeto-pitzi/arqCSV")
lista_csv

['Base CASE - Parceiros.csv',
 'Base CASE - Consultores.csv',
 'Planos.csv',
 'Base CASE - Vendas.csv']

In [111]:
parceiros = pd.read_csv("/home/michelrribeiro/virtualpython/projeto-pitzi/arqCSV/" + lista_csv[0])
consultores = pd.read_csv("/home/michelrribeiro/virtualpython/projeto-pitzi/arqCSV/" + lista_csv[1])
vendas = pd.read_csv("/home/michelrribeiro/virtualpython/projeto-pitzi/arqCSV/" + lista_csv[3])

In [112]:
parceiros.head(1), consultores.head(1), vendas.head(1)

(   Parceiro         Nome
 0         1  Lojas Marco,
    id Consultor Consultor  Parceiro   Salário
 0             1  fernanda         3  3,306.00,
    id Consultor tipo venda  qtd
 0             1          A    8)

In [113]:
parceiros = parceiros.rename(columns={'Parceiro': 'parceiro', 'Nome': 'nome'})
consultores = consultores.rename(columns={'id Consultor': 'id_consultor', 'Consultor': 'consultor', 
                                          'Parceiro': 'parceiro', 'Salário': 'salario'})
vendas = vendas.rename(columns={'id Consultor': 'id_consultor', 'tipo venda': 'tipo_de_venda'})
parceiros.head(1), consultores.head(1), vendas.head(1)

(   parceiro         nome
 0         1  Lojas Marco,
    id_consultor consultor  parceiro   salario
 0             1  fernanda         3  3,306.00,
    id_consultor tipo_de_venda  qtd
 0             1             A    8)

#### Em seguida foi feita a importação do arquivo csv criado com os dados do problema de negócio e uma coluna de lucro_por_plano foi adicionada.

In [115]:
planos = pd.read_csv("/home/michelrribeiro/virtualpython/projeto-pitzi/arqCSV/Planos.csv")
planos['lucro_por_plano'] = (planos['mensalidade'] - (planos['chance'] * planos['premio']) 
                             + (planos['chance'] * planos['taxa_adicional']))
planos

,plano,mensalidade,meses,chance,premio,taxa_adicional,lucro_por_plano
0,A,33,12,0.0417,400,30,17.571
1,B,20,12,0.0208,300,15,14.072
2,C,15,12,0.0083,250,10,13.008


#### Após a criação de todos os dataframes o próximo passo foi a inserção dos dados em tabelas do banco de dados do sqlite. Assim as perguntas poderão ser respondidas por chamadas SQL.

In [116]:
conn = sql.connect("/home/michelrribeiro/virtualpython/projeto-pitzi/arqDB/premioFacil.db")

parceiros.to_sql(name='parceiros', con=conn, if_exists='replace')
consultores.to_sql(name='consultores', con=conn, if_exists='replace')
vendas.to_sql(name='vendas', con=conn, if_exists='replace')
planos.to_sql(name='planos', con=conn, if_exists='replace')

3

#### A. Top 3 consultores com maior valor de vendas.

In [40]:
query1 = '''SELECT c.consultor as "Consultor", SUM(v.qtd * pl.mensalidade) as "Valor de Vendas" 
FROM ((planos pl JOIN vendas v ON pl.plano = v.tipo_de_venda) JOIN consultores c ON c.id_consultor = v.id_consultor)
GROUP BY c.consultor
ORDER BY "Valor de Vendas" DESC 
LIMIT 3;'''

resposta1 = pd.read_sql_query(query1, conn)
resposta1

,Consultor,Valor de Vendas
0,Natasha,57847
1,pedro,38606
2,Felipe,37104


#### B. Top 5 consultores maior número de vendas, considerando apenas vendas do plano A.

In [41]:
query2 = '''SELECT c.consultor as "Consultor", SUM(v.qtd) as "Quantidade de Vendas"
FROM ((planos pl JOIN vendas v ON pl.plano = v.tipo_de_venda) JOIN consultores c ON c.id_consultor = v.id_consultor)
WHERE v.tipo_de_venda = "A"
GROUP BY c.consultor
ORDER BY "Quantidade de Vendas" DESC
LIMIT 5;'''

resposta2 = pd.read_sql_query(query2, conn)
resposta2

,Consultor,Quantidade de Vendas
0,Natasha,739
1,Felipe,598
2,jUliA,593
3,Pedro eduardo,586
4,matheus,577


#### C. Top 2 consultores em número de vendas, dentro do parceiro com maior valor de vendas.

In [42]:
query3 = '''SELECT c.consultor as "Consultor", SUM(v.qtd) as "Quantidade de Vendas", pa.nome as "Parceiro"
FROM (((planos pl JOIN vendas v ON pl.plano = v.tipo_de_venda)
JOIN consultores c ON c.id_consultor = v.id_consultor)
JOIN parceiros pa ON c.parceiro = pa.parceiro)
WHERE c.parceiro = (SELECT pa.parceiro
FROM (((planos pl JOIN vendas v ON pl.plano = v.tipo_de_venda) 
JOIN consultores c ON c.id_consultor = v.id_consultor) 
JOIN parceiros pa ON c.parceiro = pa.parceiro)
GROUP BY pa.parceiro 
ORDER BY SUM(v.qtd * pl.mensalidade) DESC
LIMIT 1)
GROUP BY c.consultor 
ORDER BY "Quantidade de Vendas" DESC
LIMIT 2;'''

resposta3 = pd.read_sql_query(query3, conn)
resposta3

,Consultor,Quantidade de Vendas,Parceiro
0,Natasha,2663,Varejão do Renato
1,pedro,1634,Varejão do Renato
